# Exercise 3: Reading and Writing NetCDF files with Python

## Aim: Introduce the netCDF4 library in Python to read and create NetCDF4 files.

### Issues covered:

- Importing netCDF4
- Reading a NetCDF file as a Dataset instance
- Accessing Dimensions, Variables, and Attributes
- Defining Dimensions, Variables, and Attributes
- Writing a NetCDF file as a Dataset

## Let's work with the netCDF4 library to examine the contents of a data file.

Import the `netCDF4` library

In [1]:
import netCDF4

Open the file "../example_data/ggas2014121200_00-18.nc" with the netCDF4 `Dataset` method,
assign it to the `ds` variable.

In [2]:
ds = netCDF4.Dataset("../example_data/ggas2014121200_00-18.nc")

Loop through and print Dataset `variables` names, this is the ID that act like the key to access the Variable.

In [3]:
for v in ds.variables:
    print(v)

longitude
latitude
surface
time
CI
SSTK
MSL
TCC
U10
V10
SKT


From the Dataset `variables`, assign the key `SSTK` to the `sst` variable.
Access the `SSTK` variable like you would a dictionary from `ds.variables`.

- Print the `shape` and `size` of the `sst` variable
- Loop through the `dimensions` of `sst` and print the dimension name and length.
- Loop through the NetCDF attributes of `sst` and print the name and value.
(use `sst.ncattrs()` to access the attributes and `getattr(sst, {attribute name})` to get the value)

In [4]:
sst = ds.variables["SSTK"]

print(sst.shape, sst.size)

for d in sst.dimensions:
    print(d, len(d))

for attr in sst.ncattrs():
    print(f"{attr} = {getattr(sst, attr)}")

(4, 1, 256, 512) 524288
time 4
surface 7
latitude 8
longitude 9
long_name = Sea surface temperature
units = K
grid_type = gaussian
_FillValue = 2.0000000400817547e+20
source = GRIB data
name = SSTK
title = Sea surface temperature
date = 12/12/14
time = 00:00


## Let's extract some data and its related coordinate information and metadata.

Using the `sst` variable from before, take a slice of `sst` as follows and assign it the variable `arr`:

```python
sst[1, 0, 10:20, 30:35]
```

- Print what type of object `arr` is

In [5]:
arr = sst[1, 0, 10:20, 30:35]
print(type(arr))

<class 'numpy.ma.core.MaskedArray'>


Assign and print the list of `dimensions` from `sst` to the variable `dims`.
Assign the `variables` dictionary of the Dataset, `ds`, to the variables `vars`

In [6]:
dims = sst.dimensions
print(dims)

vars = ds.variables

('time', 'surface', 'latitude', 'longitude')


Now extract the slices from each Dataset variable in `vars` matching those in `arr`
(i.e. matching the values in slice `[1, 0, 10:20, 30:35]` to the values in list `dims`).

Assign them to the following variables:

- Assign `time` slice to `arr_time`
- Assign `surface` slice to `arr_level`
- Assign `latitude` slice to `arr_lats`
- Assign `longitude` slice to `arr_lons`

Print the four new variables.

In [7]:
arr_time = vars['time'][1]
print(arr_time)
arr_level = vars['surface'][0]
print(arr_level)
arr_lats = vars['latitude'][10:20]
print(arr_lats)
arr_lons = vars['longitude'][30:35]
print(arr_lons)

0.25
0.0
[82.45532  81.75363  81.05194  80.350235 79.64853  78.94681  78.245094
 77.543365 76.84164  76.13991 ]
[21.09375  21.796875 22.5      23.203125 23.90625 ]


Create an empty dictionary called `metadata`, Loop through the NetCDF Variable attributes of `sst` and copy them into this new dictionary.
Use the method as before to get name and value from `sst` and assign them to the key and value of the dictionary.

Print the dictionary.

In [8]:
metadata = dict()

for attr in sst.ncattrs():
    metadata[attr] = getattr(sst, attr)

print(metadata)

{'long_name': 'Sea surface temperature', 'units': 'K', 'grid_type': 'gaussian', '_FillValue': 2e+20, 'source': 'GRIB data', 'name': 'SSTK', 'title': 'Sea surface temperature', 'date': '12/12/14', 'time': '00:00'}


## Let's write the data/metadata from the previous section to a new NetCDF file

In this section, we will define our own Variables, Dimensions, Attributes and save them as a NetCDF file.

To start, import numpy as np.

In [9]:
import numpy as np

Open a new netCDF `Dataset` for writing to a file: "mydata.nc", specify the `mode` as write with "w" and the `format` as "NETCDF4_CLASSIC"
Assign this to the variable `myds`

In [10]:
myds = netCDF4.Dataset("mydata.nc", mode="w", format="NETCDF4_CLASSIC")

Create four new Dimensions to `myds` from your previous slices. Re-use the names from the last section.
Note that the "level" and "time" Dimensions should have length, "lat" length 10 and "lon" length 5.
To create a new Dimension use `myds.createDimension(name, size)`

In [ ]:
time = myds.createDimension('time', 1)
level = myds.createDimension('level', 1)
lat = myds.createDimension('lat', 10)
lon = myds.createDimension('lon', 5)

Create four Variables from those dimensions and assign them following this example for times:

```python
times = myds.createVariable('times', np.float64, ('time',))
```

In [ ]:
times = myds.createVariable('times', np.float64, ('time',))
levels = myds.createVariable('levels', np.float64, ('level',))
lats = myds.createVariable('lats', np.float64, ('lat',))
lons = myds.createVariable('lons', np.float64, ('lon',))

Create `myvar` as a new Dataset Variable, with id "temp", type "np.float64", and dimensions: "time", "level", "lat", "lon".

In [ ]:
myvar = myds.createVariable("temp", np.float64, ('time', 'level', 'lat', 'lon',))

Remove the `_FillValue` value in the `metadata` dictionary.
The next step will not work unless we do this.
Fill values should be handled when the Variable is created, but we are ignoring that for this example.

Use `del` to remove the `_FillValue` from `metadata`

In [14]:
del metadata['_FillValue']

Write Variable attributes from the key/value pairs found in the input data (held in the `metadata` dictionary) to `myvar`.
Use `myvar.setncattr(key, value)` to write attributes to the Dataset.

Write one more global attribute "source" to `myds`. Set the attribute `source` to the value "super dataset".
Use `myds.source` to create and set the value to the global attribute.

In [15]:
for (key, value) in metadata.items():
    myvar.setncattr(key, value)

myds.source = "super dataset"

Assign the `arr` array from before to `myvar[:]`

Write some data values to each of your four spatio-temporal variables you created.
Use simple lists of integers for these.
Make sure they are the right length matching the slices from the previous section,
use the index `[:]` on your `myds` Variables and assign the created array variables to them.

Lastly close the Dataset, `myds`, to save the file.

In [16]:
myvar[:] = arr

times[:] = arr_time
levels[:] = arr_level
lats[:] = arr_lats
lons[:] = arr_lons

myds.close()

To view the file contents, you can use the Jasmin service to run the `$ ncdump mydata.nc` command from the directory where the file is saved. Alternatively, you can call out to the linux shell from a Notebook, using: 

```
!ncdump mydata.nc
```

In [17]:
!ncdump mydata.nc

netcdf mydata {
dimensions:
	time = 1 ;
	level = 1 ;
	lat = 10 ;
	lon = 5 ;
variables:
	double times(time) ;
	double levels(level) ;
	double lats(lat) ;
	double lons(lon) ;
	double temp(time, level, lat, lon) ;
		temp:long_name = "Sea surface temperature" ;
		temp:units = "K" ;
		temp:grid_type = "gaussian" ;
		temp:source = "GRIB data" ;
		temp:name = "SSTK" ;
		temp:title = "Sea surface temperature" ;
		temp:date = "12/12/14" ;
		temp:time = "00:00" ;

// global attributes:
		:source = "super dataset" ;
data:

 times = 0.25 ;

 levels = 0 ;

 lats = 82.455322265625, 81.7536315917969, 81.0519409179688, 
    80.3502349853516, 79.6485290527344, 78.9468078613281, 78.2450942993164, 
    77.5433654785156, 76.8416366577148, 76.1399078369141 ;

 lons = 21.09375, 21.796875, 22.5, 23.203125, 23.90625 ;

 temp =
  271.459716796875, 271.459716796875, 271.459716796875, 271.459716796875, 
    271.459716796875,
  271.459716796875, 271.459716796875, 271.459716796875, 271.459716796875, 
    271.45971